In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as ss

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

In [ ]:
df = pd.read_csv("diabetes.csv")
df = df[df["BMI"]>10]
df = df[df["BloodPressure"]>10]
dfD = df[df["Outcome"]==1]
dfN = df[df["Outcome"]==0]
dPos = dfD.BMI
dNeg = dfN.BMI
df.head()

<h1>Chance - Is the Difference in BMI Between Diabetics and Non-Diabetics Legit, or Due to Chance?</h1>

<h4>We can do some testing - 

In [ ]:
#Basics
print("Non-Diabetics mean and median", dNeg.mean(), dNeg.median())
print("Diabetics mean and median", dPos.mean(), dPos.median())

In [ ]:
sns.histplot(dPos, kde=True, stat="density")
sns.histplot(dNeg, kde=True, stat="density", color='red')


<h2>OK - Looks Kind of Different, What About Chance???</h2>

Hypothesis testing can allow us to test if differences look to be real, or are likely to be determined by chance. 

This is something called a hypothesis test, and is a classic (if problematic) statistical test. Our steps will be:
<ul>
<li>Define a test statistic - whatever we are examining.
<li>Define a null hypothesis - assumption that the effect is <b>not</b> real.
<li>Compute the p-value - the probability of the null hypothesis being true. 
<li>Determine if is statistically significant - is the p-value over/under some cutoff. 
</ul><br>
Overall, if we can determine that there is an extremely small probability that the null hypothesis is true (usually <.05), we can conclude that the impact we observered is legit. 
<br><br><br>
<h2>Testing - t's, z's, sides, and samples</h2>
There are a few varieties of hypothesis testing, and the labels often overlap and intersect:
<ul>
<li>Samples - a one sample test is to determine difference from some value. A two sample tests is comparing for a difference between two samples. 
<li>Sides - a one sided test only looks for if the test score is too high or too low. A two sided test looks for either too high or too low. 
<li>Z test - when we know the population variance. 
<li>T test - when we don't know the population variance. 
</ul>
<br><br><br>
<h2>Big important note - the book examples build calculations from scratch. To do they create custom classes and functions. I'm assuming that you all haven't covered this in detail in the programming class. It isn't super complex and is worth working through, but we won't do it now. Please walk through the examples in 9.1-9.4, and then 9.5-end and try to repeat and make sense of what they are coding. We'll walk though it in a week or so. Right now, we'll use library functions and simplified versions of the book stuff to do our tests.</h2>

First, we need a test statistic. For this one it will be the BMI - we want to determine if the BMI of diabetics being higher than the non-diabetics is due to chance or not? Aka is this difference statistically significant. 

<p>This will be a two sample t-test, it will be one-sided (as I phrased it) because we are specifying that we are testing if it is larger. Time for some investigation...

<b>Null hypothesis - the increase in BMI between diabetics and nondiabetics is due to chance. </b>

In [ ]:
#Get some basic stats
meanPos, stdPos, varPos, nPos = dPos.mean(), dPos.std(), dPos.var(), dPos.count()
print(meanPos, stdPos, varPos, nPos)
meanNeg, stdNeg, varNeg, nNeg = dNeg.mean(), dNeg.std(), dNeg.var(), dNeg.count()
print(meanNeg, stdNeg, varNeg, nNeg)

The varainaces are very close, which means that the standard t-test is valid. The difference in the means is what we will tests - is that difference likely to be real, or due to chance?

In [ ]:
#Scipy ttest: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
stat, pval = ss.ttest_ind(dPos, dNeg, alternative="greater")
pval

In [ ]:
#Check for signifigance
cutoff = .05
if pval < cutoff:
    print("Reject null hypothesis - Effect appears significant")
else:
    print("Accept null hypothesis - Effect may be due to random chance")

<h3>What is happening? (this is what the book part walks through)</h3>
It is similar in concept to the CI stuff we did...
<ul>
<li>Using the data to generate random trials.
<li>Counting how many exceed the test statistic
<li>If that number is high, then it is likely to be random
</ul>

We can do a really simple one to explore...

<h1>Hypothesis Test, by hand, with simple probabilistic scenario</h1>

We flip a coin 100 times and get 55 Heads. Is the coin fair?

Null hypothesis - the difference in heads and tails being at least 10 is due to random chance. The book has a similar example that is built 'properly' with functions and classes - this is pretty much what it is doing. The one there is generalized - we can just add some definitions for the specific scenario and reuse the guts of the test instead of rewriting everything from scratch. 

In [ ]:
#Define Test Statistic
testStat = 10 #We are examining the chances that the difference from even is 10: 55 heads, 45 tails. 
#The loop below does some trials - 'flip a coin' 100 times, count up the diff between heads and tails. 
#Repeat this 100 times - that is what is in the numheads list - the 100 diffs between #heads and #tails.
#Remember - the difference part is key, we are looking for bias, so if one side shows more than the other, that's what we care about.  

numHeads = []
#Write the loop. Run 100 trials, in each one flip a coin 100 times. 
#The list above is the end result - the list of 100 differences between the #H and the #T


#The loop below 'checks' each trial, and counts them. We want to see how many of these random trials 
#exceeds our test statistic, or how many 100 flip trials have differences in totals of H/T that are at least 10. 
#If lots of trials have that much of a diff, there is good reason be think that the 55/45 split might be random. 
#If it rarely happens in random trials - it is probably due to something else - so we reject the NH!
#Change the teststat above and see, as the difference between H and T gets smaller, the more likely it is to be random.  
success = 0
#Write the loop. check each of the differences from the list above to see if it meets our criteria.
#If it does, increment the counter above. 
#The end result is that success vvaraible being the total count of that. 


p = success/100
print("P =",p)
#Check muh Null Hyp!!
cutoff = .05
if p < cutoff:
    print("Reject null hypothesis - Effect appears significant")
else:
    print("Accept null hypothesis - Effect may be due to random chance")

Above is a simplified, and non reusable version of what is going on in the book's code. It is also what happens behind the scenes in the ttest functions that we started with. 

We can do a slightly more complex one - the coin flip idea is pretty simple, what if we want to test a difference in means like the first example?

<h1>Hypothesis Test, by hand, means in two groups</h1>

NH - Is the difference in Blood Pressure between diabetics and non-diabetics significant?

In [ ]:
#What is the test statistic?
bpP, bpN = dfD["BloodPressure"].mean(), dfN["BloodPressure"].mean()
print(bpP, bpN)
bpDiff = abs(bpP-bpN)
print(bpDiff, '<---That is my test stat. There is a diff in BP, does it matter?')
dNegBP = dfN.BloodPressure
dPosBP = dfD.BloodPressure


For this one we can't really just use the probability like dice and make simple calculations. We need to do something slightly more involved. The technique we'll use is called permutation - in this contect that means we'll take all the blood pressures and mix them into one big group. 

For each of the trials, we'll then randomly shuffle them, split them into two new gorups, and count up how many times the difference between those two groups is greater than the test stat. If it is pretty common to see differences between the average blood pressure of the two random groups be large, that is an indication that the effect may be due to randomness. If it rarely happens, that is an indication that the effect is "real". 

In [ ]:
#Combines the two groups.
#Get the length of one for splitting below. 
allBp = np.hstack((dNegBP, dPosBP))
n = len(dNegBP)


In [ ]:
#Generate random splits, and add the differnce in their means to a list. 
diffs = []
#Write the loop. For each of 100 trials, shuffle the list of BP, then split it into two. 
#Check the difference in BP and add that value to the  list above. 
# The list of 100 diffs in BP is the final goal. 
#Note: there's probably many ways to do this, the easiest is likely np.random.shuffle
#and then using allBp[n:] and allBp[:n] - which will split that list at n, giving you two pieces the sizes of the original data. 
#This part is programatically kind of weird, you can reference the solution and check back, but try to make sense of it. 

#Can print the result, just to see. 
diffs


In [ ]:
#Take the differences in means from the random samples,
#Count how many times that difference exceeds our test stat. 
success = 0
#Write the loop. Just like the previous one. 


p = success/100
print("P =",p)
#Check muh Null Hyp!!
cutoff = .05
if p < cutoff:
    print("Reject null hypothesis - Effect appears significant")
else:
    print("Accept null hypothesis - Effect may be due to random chance")

In [ ]:
#The easy way - ues the scipy ttest to calculate it.


For both versions of the calculation above, the probability of the difference being due to chance is extremely low - so we reject the null hypothesis and we can conclude that diabetics and non-diabetics have a legit difference in blood pressure. 

(as an aside, one of the things that diabetes does is as your blood glucose increases, the blood itself becomes thicker. Pumping a thicker fluid increases pressure, so this result agrees with what we know. This is also a reason diabetes does damage, the thick blood can damage smaller vessels over time.)

<h2>Challenge...</h2>
Check if the difference in average GLUCOSE, BLOODPRESSURE, and BMI between people who have had a pregnancy vs people who have not is likely random or due to chance... 

Try to do one by hand, and the rest by using the scipy ttest function. 

In [ ]:
#Calculate here....

<h1>For next time...</h1>

<h2>How much can we trust these tests? We look at the POWER...(Book 9.10)</h2>

This builds on the types of errors - false positive vs false negatives, like in a confusion matrix. 

<h2>What if the data isn't normal?</h2>
The above tests have an assumption that the data is normal. What if it isn't?

For non-normal data we need a different type of test, such as Mann-Whitney. 

<h2>What if there are 3 or more groups? </h2>

We can use ANOVA - analysis of varaince. 